<a href="https://colab.research.google.com/github/DaSeeker67/RBAC/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
  for filename in filenames[:10]:
    print(os.path.join(dirname,filename))

In [24]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
!pip install -q kaggle

# Move the JSON token to ~/.kaggle and secure it
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats


Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 99% 1.06G/1.06G [00:05<00:00, 213MB/s]
100% 1.06G/1.06G [00:05<00:00, 225MB/s]


In [ ]:
from zipfile import ZipFile

with ZipFile('dogs-vs-cats.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dogs-vs-cats') # Extract to content directory

In [28]:
# Configure TensorFlow to use the GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Set TensorFlow to use the first available GPU
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [25]:
dataset_path='/content/drive/MyDrive/dogs-vs-cats'

In [26]:
train_dir= os.path.join(dataset_path,'train')
test_dir= os.path.join(dataset_path,'test')

In [27]:
train_ds= tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150,150),
    batch_size=32,
    shuffle=True,
    seed=123
)

test_ds= tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(150,150),
    batch_size=32,
    shuffle=False,
    seed=123
)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [29]:
#normalize

train_ds= train_ds.map(lambda x, y:(x/255.0,y))
test_ds= test_ds.map(lambda x, y:(x/255.0,y))

In [30]:
val_size= int(len(train_ds)*0.2)
train_size= len(train_ds)-val_size


val_ds= train_ds.take(val_size)
train_ds= train_ds.skip(val_size)

In [ ]:
model= tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3), activation='relu', padding='same', input_shape=(256,256,3)),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(32,(3,3), activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']

)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     4,194,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,204,529 (16.04 MB)

 Trainable params: 4,204,529 (16.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1
)

Epoch 1/10
351/500 ━━━━━━━━━━━━━━━━━━━━ 14:48 6s/step - accuracy: 0.5427 - loss: 0.6876

**lets try to use transfer learning technique called feature extractons**

In [31]:

from tensorflow import keras
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16


In [33]:
conv_base.trainable= False

In [32]:
conv_base= VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(150,150,3)
)

In [34]:
model= tf.keras.Sequential([
    conv_base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [35]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,812,353 (64.13 MB)

 Trainable params: 2,097,665 (8.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [36]:
train_ds= tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150,150),
    batch_size=32,
    shuffle=True,
    seed=123
)

Found 20000 files belonging to 2 classes.


In [17]:
test_ds= tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size= (256,256),
    batch_size=32,

)

Found 5000 files belonging to 2 classes.


In [37]:
def process(image,label):
  image= tensorflow.cast(image/255.,tensorflow.float32)
  return image,label

  train_ds= train_ds.map(process)
  test_ds= test_ds.map(process)

In [38]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
history= model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1

)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7214s 12s/step - accuracy: 0.9496 - loss: 0.1386 - val_accuracy: 0.5105 - val_loss: 0.6753
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 86s 137ms/step - accuracy: 0.9719 - loss: 0.0739 - val_accuracy: 0.5017 - val_loss: 0.6764
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 133s 123ms/step - accuracy: 0.9840 - loss: 0.0451 - val_accuracy: 0.5020 - val_loss: 0.6852
Epoch 4/10
540/625 ━━━━━━━━━━━━━━━━━━━━ 8s 102ms/step - accuracy: 0.9879 - loss: 0.0348